In [1]:
from IPython.display import display, HTML
import scipy.constants as sc
from scipy.optimize import fsolve
from ipywidgets import interact, widgets
%matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
})
import os
import h5py

print(os.getpid())
%cd ../

12107
/media/work/docs/codes/QuCF/scripts-py


In [2]:
# --- Launch to change cells' width ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import subprocess
import pylib.mix as mix
import pylib.qucf_read as qucf_r
import pylib.qucf_oracle as qucf_o
path_ = "/media/work/docs/codes/QuCF/simulations/tests/test-diag-matrix-MERGE-CORRECTING/"
oracle_name = "diag-matrix"
file_name_oracle_ = "circuit_OH"

In [4]:
# ---------------------------------------------------------------
# --- Read the matrix ---
# ---------------------------------------------------------------
mix.reload_module(qucf_r)

nr = 4
N = 1 << nr
D = np.identity(N)

In [41]:
# --------------------------------------------------------------
# --- Chose the case ---
# --------------------------------------------------------------
mix.reload_module(mix)
mix.reload_module(qucf_o)
A = np.zeros((N,N), dtype=complex)

# a = 1.
# b = 2.j
# c = 3.+ 6.j
# d = 4. + 8.j

# a =  6.868e-01
# b = 0.000e+00+1.742e-01j
# c = 6.657e-01+2.766e-01j
# d = 3.839e-01+8.899e-01j

# a = 4.100e-02+8.477e-01j
# b = 6.547e-01+9.038e-01j
# c = 1.351e-01+7.960e-01j
# d = 7.140e-01+6.091e-01j

# a = np.random.rand(1)[0]
# b = 1j * np.random.rand(1)[0]
# c, d = np.random.rand(2) + 1j * np.random.rand(2)

# a, b, c, d = np.random.rand(4)

# a, b, c, d = np.random.rand(4) * 1j

# a, b, c, d = np.random.rand(4) + np.random.rand(4) * 1j

# print()
# print("a = {:10.3e}".format(a))
# print("b = {:10.3e}".format(b))
# print("c = {:10.3e}".format(c))
# print("d = {:10.3e}".format(d))


# # Option 1:
# A[0,0] = a
# A[1,1] = A[2,2] = b
# for ii in range(3, 13):
#     A[ii, ii] = c
# A[13,13] = d
# A[14,14] = A[15,15] = c

# # Option 2:
# A[0,0] = A[1,1] = a
# A[2,2] = b
# for ii in range(3, 13):
#     A[ii, ii] = a
# A[13,13] = A[14,14] = A[15,15] = b


# Option 3: random matrix:
i_end = 0
while i_end < N:
    i_start = i_end
    if i_start == N-1:
        i_end = N
    else:
        i_end = np.random.randint(i_start+1, N)
    
    i_chose = np.random.randint(3)
    if i_chose == 0:
        v1 = np.random.rand(1)[0]
    if i_chose == 1:
        v1 = np.random.rand(1)[0]*1j
    if i_chose == 2:
        v1 = np.random.rand(1)[0] + np.random.rand(1)[0]*1j
    for ii in range(i_start, i_end):
        A[ii, ii] = v1
del i_end, i_start, v1


# ---
B = qucf_o.normalize_matrix_A(A, D)

# Define the circuit structure (problem-specific part):
oo_circ = qucf_o.Circuit__()
anc = qucf_o.Regs__()
inp = qucf_o.Regs__()
anc.add_reg("ar", 1, True)
inp.add_reg("r",  nr, False)
oo_circ.set_regs(inp, anc)
oo_circ.compute_N_registers()

amin.(excl. zero) value in D: 		1.000e+00
amax. value in A: 			5.283e-01
amin.(excl. zero) value in A: 		9.643e-03

amax. value in A-norm: 			5.283e-01
amin. (excl. zero)  value in A-norm: 	9.643e-03
Adding a register 'ar'...
Adding a register 'r'...


In [42]:
# -----------------------------------------------
# --- Computation of oracle angles ---
# -----------------------------------------------
mix.reload_module(mix)
mix.reload_module(qucf_o)

# Rearrange angles:
grid_sections = qucf_o.create_grid_of_sections(oo_circ, B)

# -- Create sections of sets (sets = groups):
sys_gates_ = qucf_o.SystemGates__(
    oo_circ, 
    qucf_o.create_groups(oo_circ, grid_sections)            # Option 1
#     qucf_o.create_groups_neighbor(oo_circ, grid_sections)   # Option 2
)

# Recreate the matrix using the ORIGINAL groups
print("--- Reconstruction using original groups ---")
qucf_o.compare_matrices(oo_circ, sys_gates_.reconstruct_matrix_using_GROUPS(), B)
sys_gates_.count_groups()

# # split groups:
# sys_gates_.n_split_ = 1
# sys_gates_.split_groups()

# Sort groups:
sys_gates_.sort_groups()

# Extend the gates:
sys_gates_.set_rows_limit_for_non_extension(1)
sys_gates_.extend_sorted_groups(B)

# merge groups:
sys_gates_.merge_groups()
print("--- After the merging ---")
sys_gates_.count_groups()

# preliminary correction (optional):
sys_gates_.correct_close_groups()

# add correcting groups:
sys_gates_.correct_groups(B)
print("--- After adding the correcting groups ---")
sys_gates_.count_groups()

# Recreate the matrix using the CORRECTED groups
print("--- Reconstruction using corrected groups ---")
qucf_o.compare_matrices(oo_circ, sys_gates_.reconstruct_matrix_using_GRID(), B)
sys_gates_.count_groups()

# Construct the circuit:
print("--- Circuit construction ---")
sys_gates_.construct_circuit_OH(path_, file_name_oracle_)

--- Reconstruction using original groups ---
Matrix size: 16
Max. abs. diff-real: 0.000e+00
Max. abs. diff-imag: 0.000e+00
N = 16
Number of groups: 3

--- After the merging ---
N = 16
Number of groups: 3

--- After adding the correcting groups ---
N = 16
Number of groups: 6

--- Reconstruction using corrected groups ---
Matrix size: 16
Max. abs. diff-real: 2.717e-13
Max. abs. diff-imag: 1.507e-13
N = 16
Number of groups: 6

--- Circuit construction ---
N-input-registers = 1 with [4] qubits
N-ancilla-registers = 1 with [1] qubits
Total number of input qubits: 4
Matrix size that can be encoded: 16
N-gates: 10
N-gates (assuming Rc as a single gate): 5


In [43]:
# ----------------------------------------------------------------------------------------------------
# --- Compute the circuit and compare the matrix constructed by the oracle with the original matrix ---
# -----------------------------------------------------------------------------------------------------
mix.reload_module(qucf_r)
mix.reload_module(qucf_o)

dd_res = qucf_r.read_matrix(path_, oracle_name + "_OUTPUT.hdf5") 
qucf_o.compare_matrices(oo_circ, dd_res["A"], B)

Reading the matrix from: diag-matrix_OUTPUT.hdf5
from the path: /media/work/docs/codes/QuCF/simulations/tests/test-diag-matrix-MERGE-CORRECTING/
date of the simulation:  07-28-2023 16:52:08
matrix name:  U_BE
N = 16
Max. abs. diff-real: 4.669e-12
Max. abs. diff-imag: 2.808e-13


In [18]:
# ---------------------------------------------------------------
# --- Function to print matrix subblocks ---
# ---------------------------------------------------------------
mix.reload_module(mix)
def print_submatrix(B, flag_imag = False):
    N = B.shape[0]

    
    print("\n")
    print("****** N: {:d} *************************************".format(N))
    
    max_val = np.max(np.max(np.abs(B)))
    print("max|A|: {:0.3e}".format(max_val))
    
    min_val = np.min(np.min(np.abs(B[np.nonzero(B)])))
    print("min|A|: {:0.3e}".format(min_val))
    
    # --------------------------------------------------
    Nr_print, Nc_print = N, N
    ir_start, ic_start = 0, 0
    
    B_plot = np.imag(B) if flag_imag else np.real(B)
    line_desr = "Imag part" if flag_imag else "Real part"


    print("-------------------------- {:s} -----------------------------------------".format(line_desr))
    mix.print_subblock_colored(
        B_plot, 
        ir_start, ic_start,
        Nr_print, Nc_print,
        ff=[6, 3, "f"], 
        n_in_row = Nc_print, 
        gap_be = " ", 
    )

In [19]:
# ------------------------------------------
# --- Print matrix B = A_norm/D  ---
# ------------------------------------------
print_submatrix(B, False) 
print_submatrix(B, True) 

# print_submatrix(dd_res["A-real"], False) 
# print_submatrix(dd_res["A-imag"], False) 



****** N: 16 *************************************
max|A|: 1.000e+00
min|A|: 5.000e-01
-------------------------- Real part -----------------------------------------
--- 0:   0.500  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 1:   0.000  0.500  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 2:   0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 3:   0.000  0.000  0.000  0.500  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 4:   0.000  0.000  0.000  0.000  0.500  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 5:   0.000  0.000  0.000  0.000  0.000  0.500  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 6:   0.000  0.000  0.000  0.000  0.000  0.000  0.500  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000

In [24]:
theta_y = 2.434655880435e+00
print("{:0.3e}".format(np.cos(theta_y/2.)))

3.462e-01


In [26]:
theta_z, theta_y = -4.347141368359e-02, 2.845571544992e+00
print("{:20.3e}".format(np.exp(- 1j * theta_z/2.) * np.cos(theta_y/2.)))
# print("{:20.3e}".format(d_coef * np.cos(theta_y/2.)))

1.474e-01+3.205e-03j


In [200]:
np.random.randint(2, 10)

7